In [ ]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import scipy

import sys, os

os.chdir(sys.path[0])
os.chdir('..')
from _parameters import *

In [ ]:
def load_cvsi(moment):
    cvsi_file = open(dirs['cvsi'] + f'/cvsi_{moment}.pkl', 'rb')
    tc_f = open(dirs['cvsi'] + f'/cvsi_tc_{moment}.pkl', 'rb')

    dat, avg = pickle.load(cvsi_file)
    tc = pickle.load(tc_f)

    return dat, avg, tc

In [ ]:
# Load data
resp1_dat, resp1_avg, resp1_tc = load_cvsi('resp1')
prob2_dat, prob2_avg, prob2_tc = load_cvsi('prob2')


In [ ]:
# Plot settings
plt.rcdefaults()
plt.rcParams['font.family'] = 'Helvetica Neue'
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
def error_tc(axes, data, time, color):

    avg = np.mean(data, axis=0)

    sd = np.std(data, axis=0)
    se = sd/np.sqrt(len(data))

    axes.fill_between(time, avg-se, avg+se, alpha = 0.2, facecolor = color)
    axes.plot(time, avg, color = color, linestyle = 'solid')


In [ ]:
# Time-frequency

def plot_tfr(data, vlim, extent, title):

    vmin, vmax = vlim

    fig, axes = plt.subplots(figsize = (8,4))

    # extent=(-3, 3, 3, 40)

    im = axes.imshow(data, cmap = 'RdBu_r', extent=extent, 
                     origin="lower", aspect="auto", 
                     vmin = vmin, vmax = vmax)
    
    axes.axvline(0, color='grey', linestyle=':')

    cbar = plt.colorbar(im, ticks = [vmin,0,vmax])
    cbar.ax.tick_params(labelsize = 12)

    axes.set_title(title, size = 14)

    axes.set_xlabel('Time (s)', fontsize=14)
    axes.set_ylabel('Frequency (Hz)', fontsize=14)
    axes.tick_params(axis='both', labelsize=12)

    axes.set_xlim(-1, 2)  


plot_tfr(resp1_avg['mot'], (-30, 30), (-1, 3, 3, 40), 'Response 1')
plot_tfr(prob2_avg['mot'], (-30, 30), (-3, 3, 3, 40), 'Probe 2')
    

In [ ]:
# Motor

epoch = ['Response (1)', 'Probe (2)']
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

error_tc(ax1, resp1_tc['mot'], resp1_tc['time'], color = palette['grey'])
error_tc(ax2, prob2_tc['mot'], prob2_tc['time'], color = palette['grey'])

for i, ax in enumerate([ax1, ax2]):
    ax.axhline(0, color='grey', linestyle=':')
    ax.axvline(0, color='grey', linestyle=':')

    ax.set_ylim(-20, 30)
    ax.set_xlim(-1, 2)  

    ax.set_title(epoch[i], size = 14)

    ax.set_xlabel('Time (seconds)', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)

ax1.set_ylabel('Contra vs ipsi (%)', fontsize=14)
ax2.set_ylabel('')
ax2.set_yticks([])

fig.suptitle('Motor selection (13-30Hz)', fontsize=14)

fig_fname = dirs['plot'] + '/TFR/' + 'TFR_response.svg'
fig.savefig(fig_fname, format = 'svg', dpi = 300, bbox_inches='tight')

In [ ]:
# Motor - overlayed

epoch = ['Response (1)', 'Probe (2)']
fig, ax = plt.subplots(figsize=(6, 4))

error_tc(ax, resp1_tc['mot'], resp1_tc['time'], color = palette['blue'])
error_tc(ax, prob2_tc['mot']*-1, prob2_tc['time'], color = palette['orange'])

ax.axhline(0, color='grey', linestyle=':')
ax.axvline(0, color='grey', linestyle=':')

ax.set_ylim(-30, 30)
ax.set_xlim(-1, 2)  

ax.set_title('Motor selection (13-30Hz)', size = 14)

ax.set_xlabel('Time (seconds)', fontsize=14)
ax.set_ylabel('Contra vs ipsi (%)', fontsize=14)
ax.tick_params(axis='both', labelsize=12)

ax.text(0.75, -15, 'First response', color = palette['blue'], fontsize = 14)
ax.text(0.75, -20, 'Second response', color= palette['orange'], fontsize = 14)


In [ ]:
# Motor - difference

time_sel = (prob2_tc['time'] >= -1) & (prob2_tc['time'] <= 3)
first_second = prob2_tc['mot'][:, time_sel]
first_second = first_second*-1 - resp1_tc['mot']

epoch = ['Response (1)', 'Probe (2)']
fig, ax = plt.subplots(figsize=(6, 4))

error_tc(ax, first_second, resp1_tc['time'], color = palette['grey'])

ax.axhline(0, color='grey', linestyle=':')
ax.axvline(0, color='grey', linestyle=':')

ax.set_ylim(-30, 30)
ax.set_xlim(-1, 2)  

ax.set_title('Motor selection (13-30Hz)', size = 14)

ax.set_xlabel('Time (seconds)', fontsize=14)
ax.set_ylabel('Contra vs ipsi (%)', fontsize=14)
ax.tick_params(axis='both', labelsize=12)

ax.text(0.75, -10, 'First - second', color = palette['grey'], fontsize = 14)
